# Cancer de wilsconsin

## 1.Preliminares

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import miner_a_de_datos_aprendizaje_modelos_utilidad as utils

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Fijamos la semilla

In [ ]:
random_state=27912

## 2.Carga de datos

Cargamos todos los datos

In [ ]:
filepath="../input/breast-cancer-wisconsin-data/data.csv"
index_col="id"
target="diagnosis"
data = utils.load_data(filepath, index_col, target)

Eliminamos "unnamed"

In [ ]:
data=data.drop(data.columns[31], axis='columns')

Comprobamos

In [ ]:
data.sample(5, random_state=random_state)

In [ ]:
(X, y) = utils.divide_dataset(data, target)

In [ ]:
X.sample(5, random_state=random_state)

In [ ]:
y.sample(5, random_state=random_state)

Ahora tenemos que quitar los que tengan alta correlaccion

categorias=X.columns
corrMatrix = X.corr()
x=0
aux=x
eliminado=[]
for i in categorias:
    aux=x+1
    no_encontrado=True
    while (aux<len(categorias) and no_encontrado and not(i in eliminado)  ):
        #print(eliminado)
        if(corrMatrix[i][categorias[aux]]>0.8):
            #print([corrMatrix[i][categorias[aux]]])
            no_encontrado=False
            eliminado.append(i)
            
        aux=aux+1
    x=x+1
for i in eliminado:
    X=X.drop(i,axis=1)
    #dataC_train=dataC_train.drop(i,axis=1)
print(eliminado)

Comprobamos

In [ ]:
X.sample(5, random_state=random_state)

Dividimos el conjunto de datos  entre entrenamiento y test con un holdout

In [ ]:
stratify = y
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Comprobamos:

In [ ]:
X_train.sample(5, random_state=random_state)


In [ ]:
y_train.sample(5, random_state=random_state)


In [ ]:
X_test.sample(5, random_state=random_state)


In [ ]:
y_test.sample(5, random_state=random_state)

## 3.Modelos de clasificacion supervisada

### 3.1 Vecinos mas cercanos

In [ ]:
nvecinos=10
k_neighbors_model = KNeighborsClassifier(nvecinos)

### 3.2 Arboles de decision

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)
listaAlg=[]
listaAlg.append(decision_tree_model )

### 3.4 Adaptive Boosting(AdaBoost)

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)
listaAlg.append(adaboost_model)

### 3.5 Bootstrap Aggregating (Bagging)

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)
listaAlg.append(bagging_model)

### 3.6 Random Forests

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)
listaAlg.append(random_forest_model)

### 3.7 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)
listaAlg.append(gradient_boosting_model)

### 3.8 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)
listaAlg.append(hist_gradient_boosting_model)

## 4.Evaluacion de modelos

Nuestros arboles seran de 5x10-cv

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

In [ ]:
X=X_train
y=y_train

In [ ]:
model = decision_tree_model

utils.evaluate_estimator(model, X, y, cv)

 ## 5 Seleccion de modelos

In [ ]:
resul=[]

### 5.1 k_neighbors_model

In [ ]:
estimator = k_neighbors_model
weights = ["uniform", "distance"]
nvecinos = [1, 2, 3, 4, 5, 6, 7, 8]

k_neighbors_clf1 = utils.optimize_params(estimator,
                                        X, y, cv,
                                        weights=weights,
                                        n_neighbors=nvecinos)


In [ ]:
vcn=[]
vcn.append([0.924769,7.6])

In [ ]:
estimator = k_neighbors_model
weights = ["uniform", "distance"]
nvecinos = [1, 2, 3, 4, 5, 6, 7, 8]
algorithm=["auto", "ball_tree", "kd_tree", "brute"]
k_neighbors_clf2 = utils.optimize_params(estimator,
                                        X, y, cv,
                                        weights=weights,
                                        n_neighbors=nvecinos,
                                         algorithm=algorithm
                                        )

In [ ]:
vcn.append([0.924769,33])

In [ ]:
estimator = k_neighbors_model
nvecinos = [1, 2, 3, 4, 5, 6, 7, 8]
algorithm=["auto", "ball_tree", "kd_tree", "brute"]
k_neighbors_clf3 = utils.optimize_params(estimator,
                                        X, y, cv,
                                      
                                        n_neighbors=nvecinos,
                                         algorithm=algorithm
                                        )

In [ ]:
vcn.append([0.924769,20.9])

Comprobamos los tres 

In [ ]:
vcn

Vemos que todos dan lo mismo pero con distinto tiempo por lo que escogemos el primero

In [ ]:
resul.append(k_neighbors_clf1)

### 5.2 decision_tree_model

In [ ]:

estimator = clone(decision_tree_model)

criterion = ["gini", "entropy"]
max_depth = [0, 1, 2, 3, 4, 5, 6]

ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
decision_tree_clf1 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha,
                                        )


In [ ]:
dtm=[]
dtm.append([0.943231,21.5])


In [ ]:
estimator = clone(decision_tree_model)

criterion = ["gini", "entropy"]
max_depth = [0, 1, 2, 3, 4, 5, 6]

ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
splitter=["best", "random"]
decision_tree_clf2 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha,
                                          splitter=splitter
                                        )


In [ ]:
dtm.append([0.956795,40])

Splitter vemos que funciona bien pero tarda mas asi que probamos quitando uno de los que ya estaban 

In [ ]:
estimator = clone(decision_tree_model)


max_depth = [0, 1, 2, 3, 4, 5, 6]

ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
splitter=["best", "random"]
decision_tree_clf3 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha,
                                          splitter=splitter
                                        )


In [ ]:
dtm.append([0.956795,19.7])

Acontinuacion vemos un pequeño resumen de puntuacion y segundos

In [ ]:
dtm

Vemos que el mejor caso es el ultimo y el que guardaremos

In [ ]:

resul.append(decision_tree_clf3)

### 5.3 adaboost_model

In [ ]:
estimator = adaboost_model

adaboost_model.get_params().keys()

In [ ]:
estimator = adaboost_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

adaboost_clf1 = utils.optimize_params(estimator,
                                     X, y, cv,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     base_estimator__criterion=criterion,
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha,
                                 
                                    )

In [ ]:
abm=[]
abm.append([0.969872,2.7*60])

Añadimos otra

In [ ]:
estimator = adaboost_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
algorithm=["SAMME", "SAMME.R"]
adaboost_clf2 = utils.optimize_params(estimator,
                                     X, y, cv,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     base_estimator__criterion=criterion,
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha,
                                     algorithm=algorithm
                                 
                                    )

In [ ]:
abm.append([0.973885,4.9*60])

El resusltado esta cerca del anterior de 0.94 pero tardando casi el doble de tiempo y ahora probamos quitar otra 

In [ ]:
estimator = adaboost_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]

max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
algorithm=["SAMME", "SAMME.R"]
adaboost_clf3 = utils.optimize_params(estimator,
                                     X, y, cv,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha,
                                     algorithm=algorithm
                                    )

In [ ]:
abm.append([0.970859,1.9*60])

In [ ]:
abm

Podemos observar que el mejor en puntuacion es el segundo pero tarda mucho por lo que el ultimo es el mejor ya que mejora tiempo sacrificando un poco de puntuacion

In [ ]:
resul.append(adaboost_clf3)

### 5.4 Bagging_Model

In [ ]:
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]

bagging_clf1 = utils.optimize_params(estimator,
                                    X, y, cv,
                                    base_estimator=base_estimator,
                                    base_estimator__criterion=criterion)

In [ ]:
clf=[]
clf.append([0.959269,2.9])

In [ ]:
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
warm_start=[True,False]

bagging_clf2 = utils.optimize_params(estimator,
                                    X, y, cv,
                                    base_estimator=base_estimator,
                                    base_estimator__criterion=criterion,
                                    warm_start=warm_start
                                   )

In [ ]:
clf.append([0.959269,5.9])

Añadiendo nos sale tanto peor tiempo y peor puntacion probamos quitar otra

In [ ]:
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]

warm_start=[True,False]
bagging_clf3 = utils.optimize_params(estimator,
                                    X, y, cv,
                                    base_estimator=base_estimator,
                                   
                                    warm_start=warm_start
                                   )


In [ ]:
clf.append([0.953231,5.3])

Comprobamos cual es mejor

In [ ]:
clf

Vemos que no existe ninguna mejora por lo que escogeremos el primero

In [ ]:
resul.append(bagging_clf1)

### 5.5 Random_Forest_Model

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf1 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_features=max_features)

In [ ]:
fclf=[]
fclf.append([0.964346,22.9])

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
bootstrap=[True,False]
random_forest_clf2 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_features=max_features,
                                          bootstrap=bootstrap
                                         )

In [ ]:
fclf.append([0.964346,45.8])

Aqui vemos que empeora en ambos aspectos

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
bootstrap=[True,False]
random_forest_clf3 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          
                                          max_features=max_features,
                                          bootstrap=bootstrap
                                         )

In [ ]:
fclf.append([0.962308,21.9])

Ahora comprobaremos cual es mejor

In [ ]:
fclf

Vemos quela unica mejora ,pequeña es en el tercero pero en el aspecto del tiempo mientras que baja su puntuacion por lo que nos quedamos con el primero 


In [ ]:
resul.append(random_forest_clf1)

### 5.6 Gradien_Boosting_Model

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf1 = utils.optimize_params(estimator,
                                              X, y, cv,
                                              learning_rate=learning_rate,
                                              criterion=criterion,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

In [ ]:
gdm=[]
gdm.append([0.967346,3*60])

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
max_features=["auto", "sqrt", "log2"]
gradient_boosting_clf2 = utils.optimize_params(estimator,
                                              X, y, cv,
                                              learning_rate=learning_rate,
                                              criterion=criterion,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

In [ ]:
gdm.append([0.943725,1.7*60])

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
max_features=["auto", "sqrt", "log2"]
gradient_boosting_clf3 = utils.optimize_params(estimator,
                                              X, y, cv,
                                              learning_rate=learning_rate,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

In [ ]:
gdm.append([0.942549,51.4])

Comprobamos cual es la mejor

In [ ]:
gdm

Vemos que solo mejoramos en tiempo pero no en puntuacion por lo que conservaremos el primero

In [ ]:
resul.append(gradient_boosting_clf1)

### 5.7 hist_gradient_boosting_clf

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]

hist_gradient_boosting_clf1 = utils.optimize_params(estimator,
                                                   X, y, cv,
                                                   learning_rate=learning_rate,
                                                   max_leaf_nodes=max_leaf_nodes)

In [ ]:
hgb=[]
hgb.append([0.968833,2.8*60])

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
loss=["binary_crossentropy", "categorical_crossentropy", "auto"]
hist_gradient_boosting_clf2 = utils.optimize_params(estimator,
                                                   X, y, cv,
                                                    loss=loss,
                                                   learning_rate=learning_rate,
                                                   max_leaf_nodes=max_leaf_nodes)

In [ ]:
hgb.append([0.922745,2*60])

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
loss=["binary_crossentropy", "categorical_crossentropy", "auto"]
hist_gradient_boosting_clf3 = utils.optimize_params(estimator,
                                                   X, y, cv,
                                                    loss=loss,
                                                   max_leaf_nodes=max_leaf_nodes)

In [ ]:
hgb.append([0.929804,22.9])

Comprobamos cual es el mejor

In [ ]:
hgb

Vemos que el primero tiene mejor puntacion pero que el tercero recorta mucho tiempo, pero aun asi la diferencia es de un 0.04 por lo que conservamos el tercero

In [ ]:
resul.append(hist_gradient_boosting_clf3)

## 6. Construcción y validación del modelo final


In [ ]:
estimators = {
    "Nearest neighbors": resul[0],
    "Decision tree": resul[1],
    "AdaBoost": resul[2],
    "Bagging": resul[3],
    "Random Forests": resul[4],
    "Gradient Boosting": resul[5],
    "Histogram Gradient Boosting": resul[6]
}

In [ ]:
len(resul)

In [ ]:
X = X_test
y = y_test

utils.evaluate_estimators(estimators, X, y)

# Diabetes

## 2.Carga de datos

Cargamos todos los datos

> 

In [ ]:
filepath="../input/pima-indians-diabetes-database/diabetes.csv"

target="Outcome"
data = utils.pd.read_csv(filepath, dtype={"Outcome":"category"})

In [ ]:
data.sample(5, random_state=random_state)

In [ ]:
(X, y) = utils.divide_dataset(data, target)

In [ ]:
X.sample(5, random_state=random_state)

In [ ]:
y.sample(5, random_state=random_state)

Ahora tenemos que quitar los que tengan alta correlaccion

Comprobamos

In [ ]:
X.sample(5, random_state=random_state)

Dividimos el conjunto de datos  entre entrenamiento y test con un holdout

In [ ]:
stratify = y
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Comprobamos:

In [ ]:
X_train.sample(5, random_state=random_state)


In [ ]:
y_train.sample(5, random_state=random_state)


In [ ]:
X_test.sample(5, random_state=random_state)


In [ ]:
y_test.sample(5, random_state=random_state)

## 3.Modelos de clasificacion supervisada

### 3.1 Vecinos mas cercanos

In [ ]:
nvecinos=10
k_neighbors_model = KNeighborsClassifier(nvecinos)

### 3.2 Arboles de decision

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)
listaAlg=[]
listaAlg.append(decision_tree_model )

### 3.4 Adaptive Boosting(AdaBoost)

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)
listaAlg.append(adaboost_model)

### 3.5 Bootstrap Aggregating (Bagging)

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)
listaAlg.append(bagging_model)

### 3.6 Random Forests

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)
listaAlg.append(random_forest_model)

### 3.7 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)
listaAlg.append(gradient_boosting_model)

### 3.8 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)
listaAlg.append(hist_gradient_boosting_model)

## 4.Evaluacion de modelos

Nuestros arboles seran de 5x10-cv

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

In [ ]:
X=X_train
y=y_train

In [ ]:
model = decision_tree_model

utils.evaluate_estimator(model, X, y, cv)

 ## 5 Seleccion de modelos

In [ ]:
resul=[]

### 5.1 k_neighbors_model

In [ ]:
estimator = k_neighbors_model
weights = ["uniform", "distance"]
nvecinos = [1, 2, 3, 4, 5, 6, 7, 8]

k_neighbors_clf1 = utils.optimize_params(estimator,
                                        X, y, cv,
                                        weights=weights,
                                        n_neighbors=nvecinos)


In [ ]:
vcn=[]
vcn.append([0.747575,8.4])

In [ ]:
estimator = k_neighbors_model
weights = ["uniform", "distance"]
nvecinos = [1, 2, 3, 4, 5, 6, 7, 8]
algorithm=["auto", "ball_tree", "kd_tree", "brute"]
k_neighbors_clf2 = utils.optimize_params(estimator,
                                        X, y, cv,
                                        weights=weights,
                                        n_neighbors=nvecinos,
                                         algorithm=algorithm
                                        )

In [ ]:
vcn.append([0.747575,36.1])

In [ ]:
estimator = k_neighbors_model
nvecinos = [1, 2, 3, 4, 5, 6, 7, 8]
algorithm=["auto", "ball_tree", "kd_tree", "brute"]
k_neighbors_clf3 = utils.optimize_params(estimator,
                                        X, y, cv,
                                      
                                        n_neighbors=nvecinos,
                                         algorithm=algorithm
                                        )

In [ ]:
vcn.append([0.747575,24.1])

Comprobamos los tres 

In [ ]:
vcn

Vemos que todos dan lo mismo pero con distinto tiempo por lo que escogemos el primero

In [ ]:
resul.append(k_neighbors_clf1)

### 5.2 decision_tree_model

In [ ]:

estimator = clone(decision_tree_model)

criterion = ["gini", "entropy"]
max_depth = [0, 1, 2, 3, 4, 5, 6]

ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
decision_tree_clf1 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha,
                                        )


In [ ]:
dtm=[]
dtm.append([0.755332,14.2])


In [ ]:
estimator = clone(decision_tree_model)

criterion = ["gini", "entropy"]
max_depth = [0, 1, 2, 3, 4, 5, 6]

ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
splitter=["best", "random"]
decision_tree_clf2 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha,
                                          splitter=splitter
                                        )


In [ ]:
dtm.append([0.755332,30.8])

Splitter vemos que funciona bien pero tarda mas asi que probamos quitando uno de los que ya estaban 

In [ ]:
estimator = clone(decision_tree_model)


max_depth = [0, 1, 2, 3, 4, 5, 6]

ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
splitter=["best", "random"]
decision_tree_clf3 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha,
                                          splitter=splitter
                                        )


In [ ]:
dtm.append([0.755332,17.6])

Acontinuacion vemos un pequeño resumen de puntuacion y segundos

In [ ]:
dtm

vemos que el mejor es el primero

In [ ]:

resul.append(decision_tree_clf1)

### 5.3 adaboost_model

In [ ]:
estimator = adaboost_model

adaboost_model.get_params().keys()

In [ ]:
estimator = adaboost_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

adaboost_clf1 = utils.optimize_params(estimator,
                                     X, y, cv,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     base_estimator__criterion=criterion,
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha,
                                 
                                    )

In [ ]:
abm=[]
abm.append([0.758609,1.6*60])

Añadimos otra

In [ ]:
estimator = adaboost_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
algorithm=["SAMME", "SAMME.R"]
adaboost_clf2 = utils.optimize_params(estimator,
                                     X, y, cv,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     base_estimator__criterion=criterion,
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha,
                                     algorithm=algorithm
                                 
                                    )

In [ ]:
abm.append([0.766143,2.4*60])

El resusltado esta cerca del anterior de 0.94 pero tardando casi el doble de tiempo y ahora probamos quitar otra 

In [ ]:
estimator = adaboost_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]

max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
algorithm=["SAMME", "SAMME.R"]
adaboost_clf3 = utils.optimize_params(estimator,
                                     X, y, cv,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha,
                                     algorithm=algorithm
                                    )

In [ ]:
abm.append([0.766143,1.1*60])

In [ ]:
abm

Podemos observar que el mejor en puntuacion es el segundo pero tarda mucho por lo que el ultimo es el mejor ya que mejora tiempo sacrificando un poco de puntuacion

In [ ]:
resul.append(adaboost_clf3)

### 5.4 Bagging_Model

In [ ]:
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]

bagging_clf1 = utils.optimize_params(estimator,
                                    X, y, cv,
                                    base_estimator=base_estimator,
                                    base_estimator__criterion=criterion)

In [ ]:
clf=[]
clf.append([0.750140,2])

In [ ]:
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
warm_start=[True,False]

bagging_clf2 = utils.optimize_params(estimator,
                                    X, y, cv,
                                    base_estimator=base_estimator,
                                    base_estimator__criterion=criterion,
                                    warm_start=warm_start
                                   )

In [ ]:
clf.append([0.750140,3.8])

Añadiendo nos sale tanto peor tiempo y peor puntacion probamos quitar otra

In [ ]:
estimator = bagging_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]

warm_start=[True,False]
bagging_clf3 = utils.optimize_params(estimator,
                                    X, y, cv,
                                    base_estimator=base_estimator,
                                   
                                    warm_start=warm_start
                                   )


In [ ]:
clf.append([0.75014,1.8])

Comprobamos cual es mejor

In [ ]:
clf

Vemos que el mejor caso es el ultimo pero con misma puntacion pero distinto tiempo

In [ ]:
resul.append(bagging_clf3)

### 5.5 Random_Forest_Model

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf1 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_features=max_features)

In [ ]:
fclf=[]
fclf.append([0.762774,21.4])

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
bootstrap=[True,False]
random_forest_clf2 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          criterion=criterion,
                                          max_features=max_features,
                                          bootstrap=bootstrap
                                         )

In [ ]:
fclf.append([0.762774,40.1])

Aqui vemos que empeora en ambos aspectos

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
bootstrap=[True,False]
random_forest_clf3 = utils.optimize_params(estimator,
                                          X, y, cv,
                                          
                                          max_features=max_features,
                                          bootstrap=bootstrap
                                         )

In [ ]:
fclf.append([0.758735,20])

Ahora comprobaremos cual es mejor

In [ ]:
fclf

No vemos ninguna mejora

In [ ]:
resul.append(random_forest_clf1)

### 5.6 Gradien_Boosting_Model

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf1 = utils.optimize_params(estimator,
                                              X, y, cv,
                                              learning_rate=learning_rate,
                                              criterion=criterion,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

In [ ]:
gdm=[]
gdm.append([0.761719,1.2*60])

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
max_features=["auto", "sqrt", "log2"]
gradient_boosting_clf2 = utils.optimize_params(estimator,
                                              X, y, cv,
                                              learning_rate=learning_rate,
                                              criterion=criterion,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

In [ ]:
gdm.append([0.761719,1.2*60])

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]
max_features=["auto", "sqrt", "log2"]
gradient_boosting_clf3 = utils.optimize_params(estimator,
                                              X, y, cv,
                                              learning_rate=learning_rate,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

In [ ]:
gdm.append([0.761719,36.6])

Comprobamos cual es la mejor

In [ ]:
gdm

 Vemos que la unica opcion es la tercera 

In [ ]:
resul.append(gradient_boosting_clf3)

### 5.7 hist_gradient_boosting_clf

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]

hist_gradient_boosting_clf1 = utils.optimize_params(estimator,
                                                   X, y, cv,
                                                   learning_rate=learning_rate,
                                                   max_leaf_nodes=max_leaf_nodes)

In [ ]:
hgb=[]
hgb.append([0.743124,1.5*60])

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
loss=["binary_crossentropy", "categorical_crossentropy", "auto"]
hist_gradient_boosting_clf2 = utils.optimize_params(estimator,
                                                   X, y, cv,
                                                    loss=loss,
                                                   learning_rate=learning_rate,
                                                   max_leaf_nodes=max_leaf_nodes)

In [ ]:
hgb.append([0.743124,3.1*60])

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
loss=["binary_crossentropy", "categorical_crossentropy", "auto"]
hist_gradient_boosting_clf3 = utils.optimize_params(estimator,
                                                   X, y, cv,
                                                    loss=loss,
                                                   max_leaf_nodes=max_leaf_nodes)

In [ ]:
hgb.append([0.732285,37.3])

Comprobamos cual es el mejor

In [ ]:
hgb

Vemos que el primero tiene mejor puntacion pero que el tercero recorta mucho tiempo

In [ ]:
resul.append(hist_gradient_boosting_clf3)

## 6. Construcción y validación del modelo final


In [ ]:
estimators = {
    "Nearest neighbors": resul[0],
    "Decision tree": resul[1],
    "AdaBoost": resul[2],
    "Bagging": resul[3],
    "Random Forests": resul[4],
    "Gradient Boosting": resul[5],
    "Histogram Gradient Boosting": resul[6]
}

In [ ]:
len(resul)

In [ ]:
X = X_test
y = y_test

utils.evaluate_estimators(estimators, X, y)

# Diabetes